# Coursera IBM capstone project -  The Battle of Neighborhoods

<p> project title: </p>

<h2>Finding a perfect location for opening a new restaurant that is suitable during the ‘covid - 19’ pandemic in Riyadh, Saudi Arabia using Data science and analytical tools</h2>

<p> Author: Raihan Insan Habibi</p>

<b>1. Introduction / business problems</b>

On 4th May 2020, Saudi Arabia was struck by a deadly virus called the coronavirus also known by its scientific name 'covid-19'. and on the 9th of September 2020, the recovery rate in Saudi Arabia has increased dramatically because of the new-normal systems. social distancing has changed many methods for many businesses, especially for restaurants. Since things are slowly going back to normal, finding a perfect location to open up a new restaurant would be a good idea. Finding a perfect location requires various factors that data scientists need to consider and process, as well as all of the access of data information around the neighbourhood. In this report, I will be describing a simple analysis of data from Riyadh, Saudi Arabia city neighbourhood to find a a decent location to open up a new restaurant.

<b>2. Data</b>

(all data resources will be here)

In [6]:
import numpy as np
import pandas as pd
import requests
#!pip install geopy
import geopy
#!pip install folium
import folium
print("libaries imported")

libaries imported
